In [1]:
import sys
import os
import cv2
import numpy as np
import random
import math
from matplotlib import pyplot as plt
path_to_video = './input.mp4'
%matplotlib inline

In [2]:
def SSD(current_frame, reference_frame, k, x, y, x1, y1):
    ssd = 0
    for a in range(-k,k+1):
        for b in range(-k,k+1):
            for c in range(0,3):
                ssd += (int(current_frame[x+a][y+b][c]) - int(reference_frame[x1+a][y1+b][c])) ** 2
    ssd = ssd ** 0.5
    return ssd
    

In [3]:
def find_the_vertex(current_frame, reference_frame, k, x, y, neighbour_radius, frame_height, frame_width):
    min = [];
    for x1 in range(-neighbour_radius,neighbour_radius+1):
        for y1 in range(-neighbour_radius,neighbour_radius+1):
            if(x + x1*2*k - k <= 0) or (y + y1*2*k -k  <= 0) or (x + x1*2*k +k>=frame_height) or (y + y1*2*k +k>=frame_width):
                continue
            ssd = SSD(current_frame, reference_frame, k, x, y, x + x1*2*k, y + y1*2*k)
            if len(min) == 0:
                min = [x1,y1,ssd]
            elif ssd<min[2]:
                min = [x1,y1,ssd]
    return x+min[0]*k,y+min[1]*k,min[2]

In [4]:
def find_the_vertices(current_frame, reference_frame, frame_height, frame_width, k, neighbour_radius, Tmin, Tmax):
    block_horizontal_number = int(frame_width/(2*k+1))
    block_vertical_number = int(frame_height/(2*k+1))
    img = current_frame
    for x in range(2,block_vertical_number+1):
        for y in range(2,block_horizontal_number+1):
            #print("the x is {}, y is {}".format(str(k * (x * 2 - 1)),str(k * (y * 2 - 1))))
            x1, y1, ssd = find_the_vertex(current_frame, reference_frame, k, k * (x * 2 - 1) , k * (y * 2 - 1), neighbour_radius, frame_height, frame_width)
            if (ssd < Tmin) or (ssd>Tmax) or (x1==k * (x * 2 - 1) and y1 == k * (y * 2 - 1)):
                continue
            img = arrowdraw(img,k * (y * 2 - 1),k * (x * 2 - 1),y1,x1)
    return img

In [5]:
def arrowdraw(img, x1, y1, x2, y2):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 5)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 0, 0), 5)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 0, 0), 5)
    
    return img

### Extract Frames from a Video

In [6]:
frame_save_path = './frames/'

In [7]:
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [8]:
cap = cv2.VideoCapture(path_to_video)
create_dir_if_not_exists(frame_save_path) # Or you can create it manully
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_counter = 0 # FRAME_COUNTER
while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    cv2.imshow('VideoWindowTitle-Quadrangle', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    # Main Content - End    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Video Reach End


### Convert Frame Sequence to Video

In [9]:
frame_load_path = './frames/'
path_to_output_video = './output.mov'

In [11]:

frame_counter = 0
while(1):
    current_frame = cv2.imread(frame_load_path + 'frame%d.tif' % frame_counter)
    reference_frame = cv2.imread(frame_load_path + 'frame%d.tif' %(frame_counter+1))
    if reference_frame is None:
        print('No more frames to be loaded')
        break;
    img = find_the_vertices(current_frame, reference_frame, frame_height, frame_width, 3, 2,100,200)
    frame_counter += 1
    cv2.imwrite(frame_save_path + 'Changedframe%d.tif' % frame_counter, img)
    print(frame_counter)
cv2.destroyAllWindows()

1


KeyboardInterrupt: 

In [ ]:
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 24, (int(frame_width), int(frame_height)))
frame_counter = 1
if (not out.isOpened()):
    print("writer open error")
while(1):
    frame = cv2.imread(frame_save_path + 'Changedframe%d.tif' % frame_counter)
    print(frame)
    if frame is None:
        print('No more frames to be loaded')
        break;
    out.write(frame)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()